In [1]:
!pip --quiet install pytorch-warmup
%cd /mnt/workspace/shell/

#这个cd就是我家里主机上就这样


/mnt/workspace/shell


In [2]:
from IPython.display import clear_output 
!pip --quiet install pytorch_spiking pytorch_lightning #pytorch_forecasting 
clear_output()

In [3]:
import os
import time
import sys

import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
# import pickle
import dill as pickle
import torch
import torch.nn as nn
from torch.nn.modules import TransformerEncoder, TransformerEncoderLayer, LayerNorm
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import pytorch_spiking
import pytorch_warmup as warmup
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [4]:
config = {
  "plots": {
        "show_plots": False,
        "xticks_interval": 1200,
        "color_actual": "#001f3f",
        "color_train": "#3D9970",
        "color_val": "#0074D9",
        "color_test": "#FF4136",
        "color_pred_train": "#3D9970",
        "color_pred_val": "#0074D9",
        "color_pred_test": "#FF4136",
    },
    "data": {
        "train_split_size": 0.80,
        "input_window": 30,
        "output_window": 10,
        "train_batch_size": 3,
        "eval_batch_size": 1,
        "scaler": "normal"
    }, 
    "model_transformer": {
        "feature_size": 250,
        "nhead": 10,
        "num_layers": 2,
        "dropout": 0.2,
        "out_features": 1,
        "init_range": 2, #0.5
        "lr": 0.0002, #0.0001,
        "loss": "dilate"
    },
    "paths": {
        "drive": {
            "agg_trade": {
                "train": "/content/drive/MyDrive/IP/Repos/HFTransformer/input_data/",
                "test": "/content/drive/MyDrive/IP/Repos/HFTransformer/input_data/", 
            },
            "orderbook": {
                "train": "/content/drive/MyDrive/IP/Repos/HFTransformer/input_data/",
                "test": "/content/drive/MyDrive/IP/Repos/HFTransformer/input_data/",
            },
            "models": "/content/drive/MyDrive/IP/Repos/HFTransformer/models/",
            "figures": "/content/drive/MyDrive/IP/Repos/HFTransformer/figures/",
            "utils": "/content/drive/MyDrive/IP/Repos/HFTransformer/utils/",
        },
        
        "local": {
            "agg_trade": {
                "train": "./data/input_data/",
                "test": "./data/input_data/", 
            },
            "orderbook": {
                "train": "./data/input_data/",
                "test": "./data/input_data/",
            },
            "models": "./models/",
            "figures": "./figures/",
        }
    }
}

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
drive = False
print(device)

cuda


In [6]:
#from google.colab import drive
#drive.mount('/content/drive')

## Data preparation: augmenting raw financial data

In [7]:
def augment_trade_data(df, lag, forecast_window=None):
    '''
    Augmenting input data.
    '''
    if forecast_window:
        df['lag_return'] = np.log(df['price'].shift(forecast_window)/df['price'].shift(forecast_window+1))
        return df.iloc[forecast_window+1:,:]
    if lag == 0:
        return df
    else:
        col_name = 'log_lag'+str(lag)+'_price'
        df[col_name] = np.log(df.price) - np.log(df.price).shift(lag)
        return df.iloc[lag:,:]
    
#后续会用到 别急 就是模拟了一个正常交易的时候的延迟

## Defining Transformer Model

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series.
    """
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        # padding on both ends of the time series
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))  # Permute for AvgPool1d (needs [batch, channel, seq_len])
        x = x.permute(0, 2, 1)  # Revert back to [batch, seq_len, features]
        return x

class series_decomp(nn.Module):
    """
    Series decomposition block.
    """
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean

class DLinear(nn.Module):
    """
    Decomposition-Linear (DLinear) model with output shape [batchsize, out_length].
    """
    def __init__(self, configs):
        super(DLinear, self).__init__()
        self.seq_len = configs.seq_len
        self.pred_len = configs.pred_len
  
        # Decomposition Kernel Size
        kernel_size = 25
        self.decompsition = series_decomp(kernel_size)
        self.individual = configs.individual
        self.channels = configs.enc_in

        if self.individual:
            self.Linear_Seasonal = nn.ModuleList()
            self.Linear_Trend = nn.ModuleList()
            
            for i in range(self.channels):
                self.Linear_Seasonal.append(nn.Linear(self.seq_len, self.pred_len))
                self.Linear_Trend.append(nn.Linear(self.seq_len, self.pred_len))
        else:
            self.Linear_Seasonal = nn.Linear(self.seq_len, self.pred_len)
            self.Linear_Trend = nn.Linear(self.seq_len, self.pred_len)

    def forward(self, x):
        # Input x shape: [Batch, Input length, Channels]
        seasonal_init, trend_init = self.decompsition(x)
        seasonal_init, trend_init = seasonal_init.permute(0, 2, 1), trend_init.permute(0, 2, 1)
        
        if self.individual:
            seasonal_output = torch.zeros([seasonal_init.size(0), seasonal_init.size(1), self.pred_len], dtype=seasonal_init.dtype).to(seasonal_init.device)
            trend_output = torch.zeros([trend_init.size(0), trend_init.size(1), self.pred_len], dtype=trend_init.dtype).to(trend_init.device)
            for i in range(self.channels):
                seasonal_output[:, i, :] = self.Linear_Seasonal[i](seasonal_init[:, i, :])
                trend_output[:, i, :] = self.Linear_Trend[i](trend_init[:, i, :])
        else:
            seasonal_output = self.Linear_Seasonal(seasonal_init)  # [Batch, Channels, Pred_len]
            trend_output = self.Linear_Trend(trend_init)  # [Batch, Channels, Pred_len]

        # Sum of seasonal and trend components
        output = seasonal_output + trend_output

  
        return output  # Output shape: [Batch, Pred_len]


In [32]:
class HFformer(nn.Module):
    def __init__(self, n_time_series, seq_len, output_seq_len, d_model=128, n_heads=8,
                 n_layers_encoder=6, dropout=0.1, output_dim=1, forward_dim=2048, use_mask=False, quantiles=None):
        '''
        Defining the HFformer model. Transformer encoder, linear decoder, with spiking activations.
        '''
        super(HFformer, self).__init__()
        self.device = device
        self.n_time_series = n_time_series
        self.d_model = d_model
        self.nheads = n_heads
        self.forward_dim = forward_dim
        self.dropout = dropout
        self.n_layers_encoder = n_layers_encoder
        self.seq_len = seq_len
        self.output_seq_len = output_seq_len
        self.mask_it = use_mask
        self.quantiles = quantiles
        self.output_dim = output_dim 

        self.dense_shape = nn.Linear(self.n_time_series, self.d_model)
        # spiking_activation = pytorch_spiking.SpikingActivation(nn.PReLU().to(self.device)).to(self.device)

        self.encoder_layer = TransformerEncoderLayer(self.d_model, self.nheads, self.forward_dim, self.dropout)
        self.encoder_norm = LayerNorm(self.d_model)
        self.transformer_enc = TransformerEncoder(self.encoder_layer, self.n_layers_encoder, self.encoder_norm).to(self.device)        
        self.output_dim_layer = nn.Linear(self.d_model, self.output_dim)
        # self.output_dim_layer = nn.LSTM(self.d_model, self.output_dim, 1, batch_first=False)
        if quantiles:
            self.out_length_lay = nn.Linear(self.seq_len, len(quantiles))#, 1, batch_first=True)
        else:
            self.out_length_lay = nn.Linear(self.seq_len, self.output_seq_len)#, 1, batch_first=True)
        self.mask = generate_square_subsequent_mask(self.seq_len).to(device)
        self.activation = nn.PReLU() #pytorch_spiking.SpikingActivation(nn.PReLU().to('cuda')).to('cuda')
        self.dlinear =  Model(seq_len = 1,
            pred_len = 1,
            enc_in = 38 , # number of input features
            individual = True)

    def make_embedding(self, x):
        '''
        Create embedding for model inputs.
        '''
        x = self.dense_shape(x)
        x = x.permute(1, 0, 2)
        if self.mask_it:
            x = self.transformer_enc(x, self.mask)
        else:
            x = self.transformer_enc(x)
        return x


    def forward(self, x):
        '''
        Forward method.
        '''
        x = self.dense_shape(x)
        x = x.permute(1, 0, 2)
        if self.mask_it:
            x = self.transformer_enc(x, self.mask)
        else:
            xiolk,m = self.transformer_enc(x)
        
        x = self
.        x = self.output_dim_layer(x)
        x = x.permute(1, 2, 0)
        x = self.activation(x)
        x = self.out_length_lay(x)
        if self.output_dim > 1:
            return x.permute(0, 2, 1)
        if self.quantiles:
            return x.view(-1, len(self.quantiles))
        else:
            return x.view(-1, self.output_seq_len)


def generate_square_subsequent_mask(sz):
    '''
    Generate mask.
    '''
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

#模型的定义

## Defining Loader

In [10]:
class TimeSeriesDataset(Dataset):
    '''
    Class for converting LOB data into model inputs.
    '''
    def __init__(self, x, y):
        self.x = x.astype(np.float32)
        self.y = y.astype(np.float32)
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return (self.x[idx], self.y[idx])


def prepare_data_x(data, window_size, lag):
    '''
    Windows the input data for the ML models.
    '''
    n_row = data.shape[0] - window_size + 1
    subset = data[:window_size]
    subset_mean = np.mean(subset, axis=0)
    output = np.zeros([n_row, window_size, len(subset_mean)])
    x_mean = np.zeros([n_row, len(subset_mean)])
    x_std = np.zeros([n_row, len(subset_mean)])
    for idx in range(n_row):
        subset = data[idx:idx+window_size]
        subset_mean = np.mean(subset, axis=0)
        subset_std = np.std(subset, axis=0) + 0.01
        subset_norm = (subset-subset_mean)/subset_std
        x_mean[idx,:] = subset_mean
        x_std[idx,:] = subset_std
        output[idx,:,:] = subset_norm
    x_mean = np.array(x_mean)
    x_std = np.array(x_std)
    return output[:-lag-1], output[-1], x_mean, x_std


def prepare_data_y(x, window_size, lag):
    '''
    Windows the target data for the ML models.
    '''
    output = np.zeros([len(x)-window_size-lag])
    std = 1.1*np.sqrt(lag)+lag*0.01
    for idx in range(0,len(x)-window_size-lag):
        output[idx] = np.log(x[window_size+lag-1+idx,0]/x[window_size-1+idx,0])*10_000
    output = output/std
    return output


def prepare_data(normalized_prices_train, dates_train, normalized_prices_test, dates_test, config, lag=1, plot=False):
    '''
    Returns input and target data.
    '''
    data_x, data_x_unseen, x_mean, x_std = prepare_data_x(normalized_prices_train, window_size=100, lag=lag)
    data_y = prepare_data_y(normalized_prices_train, window_size=100, lag=lag)
    split_index = int(data_y.shape[0]*0.8)
    data_x_train = data_x[:split_index]
    data_x_val = data_x[split_index:]
    data_y_train = data_y[:split_index]
    data_y_val = data_y[split_index:]

    return split_index, data_x_train, data_y_train, data_x_val, data_y_val


#这些就是来帮助把数据分成batch的

## Defining Custom Losses

In [11]:
def quantile_loss(y, y_pred, quantile):
  '''
  Computes quantile loss
  Standard quantile loss as defined in the "Training Procedure" section of
  the main TFT paper
  '''
  if quantile < 0 or quantile > 1:
    raise ValueError(
        'Illegal quantile value={}! Values should be between 0 and 1.'.format(
            quantile))

  prediction_underflow = y - y_pred
  q_loss = quantile * torch.max(prediction_underflow, torch.zeros_like(prediction_underflow)) + (
      1. - quantile) * torch.max(-prediction_underflow, torch.zeros_like(prediction_underflow))

  return torch.sum(q_loss, axis=-1)


#quantile loss在让模型有自信值很有帮助

## Defining Helper Functions

In [12]:
criterion_dict = {"MAE":nn.L1Loss, "MSE":nn.MSELoss, "QuantileLoss":quantile_loss}

def compute_loss(labels, output, src, criterion):
    '''
    Computes loss
    '''
    print(labels.shape,output.shape)
    if isinstance(output, torch.Tensor):
        if len(labels.shape) != len(output.shape):
            if len(labels.shape) > 1:
                if labels.shape[1] == output.shape[1]:
                    labels = labels.unsqueeze(2)
                else:
                    labels = labels.unsqueeze(0)
    loss = 0
    loss = criterion(output, labels.float())
    return loss


def train_step(model, opt, criterion, data_loader, takes_target, device,
                       num_targets=1, forward_params={}):
    '''
    Performs training of a single model. Runs through one epoch of the data.
    '''
    i = 0
    running_loss = 0.0
    model.train()
    for src, trg in data_loader:
        opt.zero_grad()
        if takes_target:
            forward_params["t"] = trg.to(device)
        src = src.to(device)
        trg = trg.to(device)
        
        # Ensure all tensors in forward_params are on the correct device
        # print(src.shape)
        output = model(src,**forward_params)
        output = output.squeeze()
        if num_targets == 1:
            labels = trg
        elif num_targets > 1:
            labels = trg[:, :, 0:num_targets]

        
        loss = compute_loss(labels, output, src, criterion[0])
        loss.backward()
        opt.step()
        running_loss += loss.item()
        i += 1
    total_loss = running_loss
    return total_loss


def validation(val_loader, model, criterion, device, num_targets=1):
    '''
    Computes the validation loss metrics.
    '''
    crit_losses = dict.fromkeys(criterion, 0)
    model.eval()
    labels = torch.Tensor(0).to(device)
    labels_all = torch.Tensor(0).to(device)
    output_all = torch.Tensor(0).to(device)
    with torch.no_grad():
        for src, targ in val_loader:
            output = torch.Tensor(0).to(device)
            src = src if isinstance(src, list) else src.to(device)
            targ = targ if isinstance(targ, list) else targ.to(device)
            output = model(src.float())
            output = output.squeeze()
            output_all = torch.cat((output_all, output))
            if num_targets == 1:
                labels = targ
            elif num_targets > 1:
                labels = targ[:, :, 0:num_targets]
            for crit in criterion:
                loss = compute_loss(labels, output, src, crit)
                crit_losses[crit] += loss.item()
            labels_all = torch.cat((labels_all, labels))
    return list(crit_losses.values())[0], output_all, labels_all
def forecast(data_loader, model, criterion, forecast_horizon, device, num_targets=1):
    '''
    Forecasting
    '''
    crit_losses = dict.fromkeys(criterion, 0)
    model.eval()
    output_decoder = torch.Tensor(0).to(device)
    labels = torch.Tensor(0).to(device)
    labels_all = torch.Tensor(0).to(device)
    counter = 0
    with torch.no_grad():
        for src, targ in data_loader:
            if (counter % forecast_horizon) == 0:
                src = src if isinstance(src, list) else src.to(device)
                targ = targ if isinstance(targ, list) else targ.to(device)
                output = model(src.float())
                #output = output.reshape(1,-1)
                output_decoder = torch.cat((output_decoder, output))
                if num_targets == 1:
                    labels = targ
                elif num_targets > 1:
                    labels = targ[:, :, 0:num_targets]
                for crit in criterion:
                    loss = compute_loss(labels, output, src, crit)
                    crit_losses[crit] += loss.item()
                labels_all = torch.cat((labels_all, labels))
            counter += 1
    return list(crit_losses.values())[0], output_decoder, labels_all


#这些属于常规操作了

## Defining Trainer

In [13]:
def strategy_evaluator(true, pred):
    '''
    Evaluates strategy regarding correct buys and sells
    '''
    total_buys, total_sells, total_holds = np.sum(true>0), np.sum(true<0), np.sum(true==0)
    total_correct_buys, total_correct_sells, total_correct_holds = 0, 0, 0
    for idx in range(len(true)):
        for jdx in range(len(true[0])):
            if true[idx,jdx] > 0 and pred[idx,jdx] > 0:
                total_correct_buys += 1
            elif true[idx,jdx] < 0 and pred[idx,jdx] < 0:
                total_correct_sells += 1
            elif true[idx,jdx] == 0 and pred[idx,jdx] == 0:
                total_correct_holds += 1
    total_correct_buys_r, total_correct_sells_r, total_correct_holds_r = (total_correct_buys/total_buys),(total_correct_sells/total_sells),(total_correct_holds/total_holds)
    return total_correct_buys_r.round(3), total_correct_sells_r.round(3), total_correct_holds_r.round(3)

In [14]:
def trainer(model, train_loader, validation_loader, test_loader, criterion, opt, scheduler,
            warmup_scheduler, max_epochs, batch_size, forecast_horizon, takes_target, shuffle=False,
            num_targets=1, plot_prediction=True, save_path='/mnt/workspace/shell/results_Transencwithlineardec'
, LAG=0):
    '''
    Training method
    '''
    start_time = time.time()
    
    data_loader = DataLoader(train_loader, batch_size=batch_size, shuffle=shuffle, sampler=None, batch_sampler=None, num_workers=10)
    validation_data_loader = DataLoader(validation_loader, batch_size=batch_size, shuffle=False, sampler=None, batch_sampler=None, num_workers=10)
    test_data_loader = DataLoader(test_loader, batch_size=1, shuffle=False, sampler=None, batch_sampler=None, num_workers=2)
    forecast_data_loader = DataLoader(validation_loader, batch_size=1, shuffle=False, sampler=None, batch_sampler=None, num_workers=2)
    
    for epoch in range(1, max_epochs+1):

        total_loss = train_step(model, opt, criterion, data_loader, takes_target, device, num_targets=num_targets)
        val_loss = 0
        if plot_prediction:
            val_loss, val_values, true_values = forecast(forecast_data_loader, model, criterion, forecast_horizon=forecast_horizon,
                                                                   device=device, num_targets=num_targets)
            fig, ax = plt.subplots(1, 1, figsize = (18, 8))
            ax.plot(true_values.cpu().view(-1), label='truth', alpha=0.3)
            ax.plot(val_values.cpu().view(-1), label='forecast', alpha=0.8)
            ax.set_xlim(left=0, right=len(true_values.cpu().view(-1)))
            plt.show()
        else:
            val_loss, val_values, true_values = validation(validation_data_loader, model, criterion, device,
                                                            num_targets=num_targets)
            # val_loss, val_values, true_values, src_all
        preds, trues = val_values.cpu().numpy(), true_values.cpu().numpy()#, src_all.cpu().numpy()

        print(f'preds {preds.shape}')
        print(f'trues {trues.shape}')

        results = 0
      
        r2_sklearn = r2_score(trues, preds)

        elapsed = time.time() - start_time
        print('-' * 88)
        print('| epoch {:3d} | {:5.2f} s | train loss {:5.5f} | val loss {:5.5f} | lr {:1.8f} | r2 sklearn: {:1.5f} | b, s, h: {:}|'.format(
                        epoch, elapsed, total_loss, val_loss, scheduler.get_last_lr()[0], r2_sklearn, results))
        print('-' * 88)
        start_time = time.time()

        if save_path:
            results = {
                    'model': 'Dlinear',
                    'pred_len': forecast_horizon,
                    'epoch': epoch,
                    'train_loss': total_loss,
                    'val_loss': val_loss,
                    'r2_val_sklearn': r2_sklearn            
            }

            df = pd.DataFrame([results])
            df.to_csv(os.path.join(save_path, 'results.csv'), mode='a', header=not os.path.exists(save_path), index=False)
            save_directory = os.path.join(save_path, "s4")
            if not os.path.exists(save_directory):
                os.makedirs(save_directory)
            if r2_sklearn >0.02 :
                torch.save(model.state_dict(), os.path.join(save_path,"s4",f'_epoch_{epoch}_time_{time.time()}_r2_{r2_sklearn}.pt'))

        with warmup_scheduler.dampening():
            scheduler.step()


## Model and Training

## Optimal paramater search

In [15]:
# date_train = 'all' 
# date_test = 'all'
date_train = 'All_to_Sept'
date_test = 'All_to_Sept'

idx = 0
agg_trade = pd.read_csv(config["paths"]["local"]["agg_trade"]["train"]+date_train+'/orderbook_agg_trade_dollarvol.csv')
agg_trade['w_midprice'] = (agg_trade['ask1']*agg_trade['askqty1']+agg_trade['bid1']*agg_trade['bidqty1'])/(agg_trade['askqty1']+agg_trade['bidqty1'])

In [33]:
model_name = 'Dlinear_tmp_1_withtransformer'

save_path = os.path.join(f'/mnt/workspace/shell/tmp/{model_name}/training_details/Dlinear/results_Dlinear',
                            str(int(time.time()))+'_results.csv')

# save_path = f'/mnt/workspace/shell/tmp/{model_name}/training_details/HFTransformer/results_HFformer'
# # save_path=None
# save_path = f'/home/gaen/Documents/codespace-gaen/Simons/{model_name}/training_details/HFTransformer/results_HFformer'
filepath = f'/mnt/workspace/shell/tmp/{model_name}/training_details/s4/results_Dlinear/Dlinear'
# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)
os.makedirs(filepath, exist_ok=True)

forecast_history = 100
epochs = 32
batch_size = 256 #64 for linear decoder

# forecast_windows = [i for i in range(1,32)]
forecast_windows = [5, 8, 10, 13, 14, 15, 16, 24, 25, 26, 28, 29, 30]
for forecast_window in forecast_windows:
    
    # orderbook = augment_trade_data(agg_trade, lag=0, forecast_window=forecast_window)
    orderbook = augment_trade_data(agg_trade, lag=0, forecast_window=forecast_window)
    # total_orw = orderbook.shape[0]
    # print(total_orw)
    features = ['price', 'lag_return',
                'bid1', 'bidqty1', 'bid2', 'bidqty2', 'bid3', 'bidqty3', 'bid4', 'bidqty4', 'bid5', 'bidqty5',
                'bid6', 'bidqty6', 'bid7', 'bidqty7', 'bid8', 'bidqty8', 'bid9', 'bidqty9',
                'ask1', 'askqty1', 'ask2', 'askqty2', 'ask3', 'askqty3', 'ask4', 'askqty4', 'ask5', 'askqty5',
                'ask6', 'askqty6', 'ask7', 'askqty7', 'ask8', 'askqty8', 'ask9', 'askqty9']


    split_index, data_x_train, data_y_train, data_x_val, data_y_val = prepare_data(np.array(orderbook[features][1_350_000:1_350_500]),
                                                                                                                            np.array(agg_trade.datetime[899_999:1_000_000]),
                                                                                                                            np.array(orderbook[features][60_000:60_600]),
                                                                                                                            np.array(agg_trade.datetime[60_000:60_600]),
                                                                                                                            config, lag=forecast_window, plot=False)

    # data_x_train_shape = data_x_train.shape[1]
    # data_y_train_shape = data_y_train.shape[1]
    # data_x_val_shape = data_x_val.shape[1]
    # data_y_val_shape = data_y_val.shape[1]
    # print( data_x_train_shape , data_y_train_shape ,data_x_val_shape, data_y_val_shape)
    train_loader = TimeSeriesDataset(data_x_train, data_y_train)
    val_loader = TimeSeriesDataset(data_x_val, data_y_val)
    test_loader = None

    model_custom = HFformer(n_time_series=len(features), seq_len=forecast_history, output_seq_len=1, d_model=64,
                  n_heads=8, n_layers_encoder=2, dropout=0.3, output_dim=1, forward_dim=64, use_mask=True).to(device)


    criterion = nn.MSELoss(reduction='sum')
    optimizer = optim.AdamW(model_custom.parameters(), lr=0.1, amsgrad=True)
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)
    warmup_scheduler = warmup.LinearWarmup(optimizer, warmup_period=1000)

    trainer(model_custom, train_loader, val_loader, test_loader, [criterion], optimizer, scheduler, warmup_scheduler, epochs, batch_size=batch_size,
        forecast_horizon=forecast_window, takes_target=False, plot_prediction=False, save_path=save_path, LAG=forecast_window)
    
    del data_x_train 
    del data_y_train
    del data_x_val
    del data_y_val

state = {
    'model': model.state_dict(),
    'epoch': forecast_window,
}
torch.save(state, f'/mnt/workspace/shell/tmp/{model_name}/Dlinear/Dlinear_forecasting_FINAL_horizon_{forecast_window}.pt')
    # torch.save(model_custom, f'/mnt/workspace/shell/tmp/{model_name}/s4_enclinear_forecasting_FINAL_horizon_{forecast_window}.pt')
print(f'Done with prediction len {forecast_window}.')


/mnt/software/anaconda3/envs/python3.10/lib/python3.10/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
/mnt/software/anaconda3/envs/python3.10/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


torch.Size([100, 256, 64])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (100x256 and 100x256)

In [ ]:
if False:
    date_train = 'all' 
    date_test = 'all'
    drive = None
    if drive:
        agg_trade = pd.read_csv(config["paths"]["drive"]["agg_trade"]["train"]+date_train+'/orderbook.csv')    
        sys.path.append(config["paths"]["drive"]["utils"])
    else:
        agg_trade = pd.read_csv(config["paths"]["local"]["agg_trade"]["train"]+date_train+'/orderbook_agg_trade_dollarvol_drop_near_duplicate_price.csv')
        agg_trade_test = pd.read_csv(config["paths"]["local"]["agg_trade"]["test"]+date_test+'/orderbook_agg_trade_dollarvol_drop_near_duplicate_price.csv')
    idx = 0
    agg_trade['w_midprice'] = (agg_trade['ask1']*agg_trade['askqty1']+agg_trade['bid1']*agg_trade['bidqty1'])/(agg_trade['askqty1']+agg_trade['bidqty1'])

    #agg_trade['price'] = agg_trade['w_midprice']
    agg_trade_test = agg_trade[4_500_000:]

In [ ]:
if False:
    save_path = os.path.join('./home/gaen/Documents/codespace-gaen/Simons/models_drop_duplicate_near/training_details/HFTransformer/results_HFformer',
                                str(int(time.time()))+'_results.csv')

    save_path = '/home/gaen/Documents/codespace-gaen/Simons/models_drop_duplicate_near/training_details/HFTransformer/results_HFformer'
    # save_path=None

    forecast_history = 100 
    epochs = 16
    batch_size = 300 #64 for linear decoder

    forecast_windows = [i for i in range(1,31)]

    for forecast_window in forecast_windows:
        
        orderbook = augment_trade_data(agg_trade, lag=0, forecast_window=forecast_window)

        features = ['price', 'lag_return',
                    'bid1', 'bidqty1', 'bid2', 'bidqty2', 'bid3', 'bidqty3', 'bid4', 'bidqty4', 'bid5', 'bidqty5',
                    'bid6', 'bidqty6', 'bid7', 'bidqty7', 'bid8', 'bidqty8', 'bid9', 'bidqty9',
                    'ask1', 'askqty1', 'ask2', 'askqty2', 'ask3', 'askqty3', 'ask4', 'askqty4', 'ask5', 'askqty5',
                    'ask6', 'askqty6', 'ask7', 'askqty7', 'ask8', 'askqty8', 'ask9', 'askqty9']

        split_index, data_x_train, data_y_train, data_x_val, data_y_val = prepare_data(np.array(orderbook[features][1_000_000-700000:1_720_000-700000]),
                                                                                                                                np.array(agg_trade.datetime[2_005_000-1200000:2_006_000-1200000]),
                                                                                                                                np.array(orderbook[features][60_000:60_600]),
                                                                                                                                np.array(agg_trade.datetime[60_000:60_600]),
                                                                                                                                config, lag=forecast_window, plot=False)


        train_loader = TimeSeriesDataset(data_x_train, data_y_train)
        val_loader = TimeSeriesDataset(data_x_val, data_y_val)
        test_loader = None

        model_custom = HFformer(n_time_series=len(features), seq_len=forecast_history, output_seq_len=1, d_model=36,
                    n_heads=6, n_layers_encoder=2, dropout=0.3, output_dim=1, forward_dim=64, use_mask=True).to(device)

        criterion = nn.MSELoss(reduction='sum')
        optimizer = optim.AdamW(model_custom.parameters(), lr=0.1, amsgrad=True)
        scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)
        warmup_scheduler = warmup.LinearWarmup(optimizer, warmup_period=1000)

        trainer(model_custom, train_loader, val_loader, test_loader, [criterion], optimizer, scheduler, warmup_scheduler, epochs, batch_size=batch_size,
            forecast_horizon=forecast_window, takes_target=False, plot_prediction=False, save_path=save_path, LAG=forecast_window)
        
        del data_x_train 
        del data_y_train
        del data_x_val
        del data_y_val

        torch.save(model_custom, f'./models_drop_duplicate_near/transformer_enclinear_forecasting_FINAL_horizon_{forecast_window}.pt')
        print(f'Done with prediction len {forecast_window}.')

## Forecast Evaluator

In [ ]:
def strategy_evaluator(true, pred, weighted=False):
    '''
    Evaluates trading strategy based on correct buys and sells.
    '''
    total_buys, total_sells, total_holds = np.sum(true>0), np.sum(true<0), np.sum(true==0)
    total_correct_buys, total_correct_sells, total_correct_holds = 0, 0, 0
    for idx in range(len(true)):
        if true[idx] > 0 and pred[idx] > 0:
            total_correct_buys += 1
        elif true[idx] < 0 and pred[idx] < 0:
            total_correct_sells += 1
        elif true[idx] == 0 and pred[idx] == 0:
            total_correct_holds += 1
    total_correct_buys_r, total_correct_sells_r, total_correct_holds_r = (total_correct_buys/total_buys),(total_correct_sells/total_sells),(total_correct_holds/total_holds)
    total_correct_trades = (total_correct_buys+total_correct_sells+total_correct_holds)/(total_buys+total_sells+total_holds)
    return total_buys, total_correct_buys, total_sells, total_correct_sells, total_holds, total_correct_holds


def forecast_evaluator(test_loader, model, criterion, forecast_horizon=1, device=device, num_targets=1, save_path=None):
    '''
    Outputs evaluation metrics.
    '''
    test_data_loader = DataLoader(test_loader, batch_size=128, shuffle=False, sampler=None, batch_sampler=None, num_workers=6)
    loss, pred, true = forecast(test_data_loader, model, criterion, forecast_horizon=1, device=device, num_targets=1)
    pred, true = pred.cpu().numpy(), true.cpu().numpy()

    r2 = r2_score(true, pred)
    strategy_results = strategy_evaluator(true, pred)
    
    if save_path:
        results = {
                'model': 'Transencwithlineardec',
                'pred_len': forecast_horizon,
                'test_loss': loss,
                'r2_val_sklearn': r2,
                'correct_buys': strategy_results[1],
                'total_buys':  strategy_results[0],
                'correct_sells': strategy_results[3],
                'total_sells': strategy_results[2],
                'correct_holds': strategy_results[5],
                'total_holds': strategy_results[4],
        }
        os.makedirs(save_path_model, exist_ok=True)
      
        df.to_csv(save_path, mode='a', header=not os.path.exists(save_path), index=False)

    print(f'| test loss {loss} | b, cb, s, cs, h, ch: {strategy_results} |')

    return pred, true


## Forecasting

In [ ]:
date_train = 'all' 
date_test = 'all'
drive  = None
if drive:
    agg_trade = pd.read_csv(config["paths"]["drive"]["agg_trade"]["train"]+date_train+'/orderbook.csv')    
    sys.path.append(config["paths"]["drive"]["utils"])
else:
    agg_trade = pd.read_csv(config["paths"]["local"]["agg_trade"]["train"]+date_train+'/orderbook_agg_trade_dollarvol.csv')
    agg_trade_test = pd.read_csv(config["paths"]["local"]["agg_trade"]["test"]+date_test+'/orderbook_agg_trade_dollarvol.csv')



In [ ]:
agg_trade_test = agg_trade[1_200_000:]
print(agg_trade_test.shape)

In [ ]:
save_path = os.path.join('/mnt/workspace/shell/models/training_details/HFTransformer/results_HFformer_tmp',
                            str(int(time.time()))+'_forecasting_results.csv')

# save_path = None

save_path_results = os.path.join('/mnt/workspace/shell/models/training_details/HFTransformer/results_HFformer_tmp',
                            str(int(time.time()))+'_list_results.pkl')

save_path_model = os.path.join('/mnt/workspace/shell/models/training_details/HFTransformer/results_HFformer_tmp',str(int(time.time()))+'_model.pth')

forecast_history = 100 
batch_size = 256 #64 for linear decoder

forecast_windows = [i for i in range(1,32)]

predictions = []

for forecast_window in forecast_windows:
    
    orderbook = augment_trade_data(agg_trade_test, lag=0, forecast_window=forecast_window)

    features = ['price', 'lag_return',
                'bid1', 'bidqty1', 'bid2', 'bidqty2', 'bid3', 'bidqty3', 'bid4', 'bidqty4', 'bid5', 'bidqty5',
                'bid6', 'bidqty6', 'bid7', 'bidqty7', 'bid8', 'bidqty8', 'bid9', 'bidqty9',
                'ask1', 'askqty1', 'ask2', 'askqty2', 'ask3', 'askqty3', 'ask4', 'askqty4', 'ask5', 'askqty5',
                'ask6', 'askqty6', 'ask7', 'askqty7', 'ask8', 'askqty8', 'ask9', 'askqty9']
    print(orderbook[features].shape)

    split_index, data_x_train, data_y_train, data_x_val, data_y_val = prepare_data(np.array(orderbook[features][:]),
                                                                                    np.array(agg_trade.datetime[2_005_000-500000:2_006_00-5000000]),
                                                                                    np.array(orderbook[features][60_000:60_600]),
                                                                                    np.array(agg_trade.datetime[60_000:60_600]),
                                                                                    config, lag=forecast_window, plot=True)

    test_loader = TimeSeriesDataset(data_x_train, data_y_train)

    model = torch.load(f'/mnt/workspace/shell/tmp/HFfMODELsept10test_tmp_1/transformer_enclinear_forecasting_FINAL_horizon_{forecast_window}.pt')
    criterion = nn.MSELoss(reduction='sum')

    pred, true = forecast_evaluator(test_loader, model, [criterion], forecast_horizon=forecast_window, device=device, num_targets=1, save_path=save_path)
    
    predictions.append((pred, true))

    print(f'Done with prediction len {forecast_window}.')

with open(save_path_results, 'wb') as f:
    pickle.dump(predictions, f)